In [2]:
import pandas as pd
from pulp import *

In [6]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.4 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [3]:
stock = pd.read_csv("../Data/data/inventory.csv")
predicted_demand = pd.read_csv("../data/forecasted_demand.csv")
transport_cost = pd.read_csv("../Data/data/transport_costs.csv")

In [4]:
date = stock['date'].max()
stock_today = stock[stock['date'] == date].groupby('warehouse')['stock_level'].sum().to_dict()
demand_today = predicted_demand[predicted_demand['date'] == date].groupby('warehouse')['predicted_demand'].sum().to_dict()
cost_dict = transport_cost.set_index(['from_warehouse', 'to_warehouse'])['cost_per_unit'].to_dict()

# Define LP model
model = LpProblem("SupplyChainOptimizer", LpMinimize)

warehouses = list(stock_today.keys())
transfers = [(i, j) for i in warehouses for j in warehouses if i != j]

# Decision variables
x = LpVariable.dicts("Transfer", transfers, lowBound=0, cat='Integer')

# Objective: Minimize total transfer cost
model += lpSum([x[i] * cost_dict[i] for i in transfers])

# Constraints: Don’t overdraw stock, meet demand
for w in warehouses:
    outgoing = lpSum([x[(w, j)] for j in warehouses if w != j])
    incoming = lpSum([x[(i, w)] for i in warehouses if i != w])
    net_stock = stock_today.get(w, 0) + incoming - outgoing
    demand_needed = demand_today.get(w, 0)
    model += net_stock >= demand_needed

# Solve
model.solve()
print("Status:", LpStatus[model.status])
for var in model.variables():
    if var.varValue > 0:
        print(var.name, "=", var.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/1ed662d883a34f60a3bb1d6445737323-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/1ed662d883a34f60a3bb1d6445737323-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 799 RHS
At line 813 BOUNDS
At line 970 ENDATA
Problem MODEL has 13 rows, 156 columns and 312 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 156 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 13 rows, 156 columns (156 integer (0 of which binary)) and 312 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0012I Integer solution of -0 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I 